**NOTE: copied and adapted from https://github.com/mkneierV/test_notebooks/blob/master/demo_data_prep.ipynb**

# Setup

In [ ]:
# default parameters

RUN_LOCAL = True
ROOT_DIR = 'temp/papermill_demo'
PROJECT = 'kubeflow-demo-256908'
BUCKET = 'gs://{}-dev'.format(PROJECT)
REGION = 'us-central1'

In [ ]:
%%capture

!pip install matplotlib --upgrade
!pip install pandas --upgrade
!pip install apache-beam[gcp]==2.16.0
!pip install six==1.12.0

In [ ]:
from datetime import datetime
import logging
import os

import apache_beam as beam
import pandas as pd
import tensorflow as tf


logging.getLogger().setLevel(logging.INFO)

In [ ]:
%load_ext google.cloud.bigquery

# Data exploration

In [ ]:
%%bigquery df

SELECT
    weight_pounds,
    is_male,
    mother_age,
    mother_race,
    plurality,
    gestation_weeks
FROM
    publicdata.samples.natality
WHERE year BETWEEN 1980 AND 2004
    AND weight_pounds > 0
    AND mother_age > 0
    AND plurality > 0
    AND gestation_weeks > 0
    AND month > 0
LIMIT 100

In [ ]:
%matplotlib inline

df['gestation_weeks'].hist();

# Helper functions

In [ ]:
FEATURES = ["weight_pounds", "is_male", "mother_age", "mother_race", "plurality", "gestation_weeks"]


def get_source_query(step):
    train_years = (1980,2004)
    eval_years  = (2005,2007)
    test_years  = (2008, 2008)
    
    query = """
    SELECT
      weight_pounds,
      is_male,
      mother_age,
      mother_race,
      plurality,
      gestation_weeks
    FROM
      publicdata.samples.natality
    WHERE year BETWEEN {} AND {}
      AND weight_pounds > 0
      AND mother_age > 0
      AND plurality > 0
      AND gestation_weeks > 0
      AND month > 0
    LIMIT 100
    """
    
    if step == 'eval':
        source_query = query.format(*eval_years)
    elif step == 'test':
        source_query = query.format(*test_years)
    elif step == "train":
        source_query = query.format(*train_years)
    else:
        raise ValueError("step value of {} must be one of 'train', 'eval', 'test'".format(step))
    return source_query



def prep_bq_row(bq_row):
    # modify opaque numeric race code into human-readable data
    races = dict(zip([1,2,3,4,5,6,7,18,28,39,48],
                     ['White', 'Black', 'American Indian', 'Chinese', 
                      'Japanese', 'Hawaiian', 'Filipino',
                      'Asian Indian', 'Korean', 'Samaon', 'Vietnamese']))
    result = {} 
    
    for feature_name in bq_row.keys():
        result[feature_name] = str(bq_row[feature_name])

    if 'mother_race' in bq_row and bq_row['mother_race'] in races:
        result['mother_race'] = races[bq_row['mother_race']]
    else:
        result['mother_race'] = 'Unknown'

    return result


def to_csv_string(bq_dict):
    output = []
    for f in FEATURES:
        output.append(bq_dict[f])
        
    return ",".join(output)

# Define and run distributed pipeline

In [ ]:
OUTPUT_DIR = os.path.join(BUCKET, ROOT_DIR)
TRANSFORMED_DATA_DIR = os.path.join(OUTPUT_DIR,'transformed')

pipeline_options = beam.pipeline.PipelineOptions(flags=[], **{
    'runner': 'DirectRunner' if RUN_LOCAL == True else 'DataflowRunner',
    'project': PROJECT,
    'region': REGION,
    'temp_location': os.path.join(OUTPUT_DIR, 'tmp'),
})


with beam.Pipeline(options=pipeline_options) as pipeline:            
    for step in ("train", "eval", "test"):
        source_query = get_source_query(step)
        data = (
            pipeline
            | '{} - Read Data from BigQuery'.format(step) >> beam.io.Read(
                beam.io.BigQuerySource(query=source_query, use_standard_sql=True))
            | '{} - Clean up Data'.format(step) >> beam.Map(prep_bq_row)
            | '{} - Prepare for csv'.format(step) >> beam.Map(to_csv_string)
            | '{} - Write Transformed Data'.format(step) >> beam.io.WriteToText(
                file_path_prefix=os.path.join(TRANSFORMED_DATA_DIR, step, step),
                file_name_suffix=".csv")
        )

In [ ]:
!gsutil ls {TRANSFORMED_DATA_DIR}

In [ ]:
pd.read_csv(
    os.path.join(TRANSFORMED_DATA_DIR, 'eval', 'eval-00000-of-00001.csv'),
    names=[
        'weight_pounds',
        'is_male',
        'mother_age',
        'mother_race',
        'plurality',
        'gestation_weeks'
    ]
)